In [11]:
import openai
from openai import OpenAI
import os 
import config 

os.environ["OPENAI_API_KEY"] = config.openai_api_key

patient_instructions = "You are a patient chatting with a doctor over an online chat interface. The doctor has never met you before. \
This is your profile: \
55-year-old female with a 3-month history of numbness and tingling in the thumb, index, middle, and ring fingers of the right hand. \
The numbness is worse at night and wakes you up from sleep. You also have pain in the hand and wrist that radiates up the forearm, \
worse with activities like typing, using a mouse, or gripping objects. \
Past Medical History includes hypertension and hypothyroidism. \
You work as a data entry clerk. Respond to the doctor's questions honestly as they interview you, asking any questions that may come up. \
You should answer as the patient. The next message in the chat will be a doctor asking you questions. \
Do not reveal you are an AI chatbot."

doctor_instructions = "You are an empathetic clinician asking a patient about their medical history over an online chat interface. \
You know nothing about the patient in advance. Respond to the patient with a single-turn response to better understand their history and symptoms. \
Do not ask more than two questions. If the patient asks a question, be sure to answer it appropriately."


In [ ]:
# psuedo code
- create function which generates a response


In [19]:
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hello!"}
  ]
)

print(completion.choices[0].message)
print(completion.choices[0].message.content)


ChatCompletionMessage(content='Hello! How can I assist you today?', role='assistant', function_call=None, tool_calls=None)
Hello! How can I assist you today?


In [23]:
completion.choices[-1].message.content

'Hello! How can I assist you today?'

In [24]:
def generate_response(messages, agent_name):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages
    )
    return {"role": "user", "content": response.choices[-1].message.content, "name": agent_name}

In [25]:
conversation_context = [
    {"role": "system", "content": doctor_instructions},
    {"role": "system", "content": patient_instructions}
]

In [28]:
from openai import OpenAI

# Initialize OpenAI client
client = OpenAI()

# Define instructions for each agent


# Initialize agents' first messages
doctors_first_message = {"role": "user", "content": "Hello, how are you?", "name": "Doctor"}
conversation_context.append(doctors_first_message)

# Conduct the conversation
def chat_between_agents(conversation_context, max_turns=5):
    for _ in range(max_turns):
        # Agent 1's turn (Doctor)
        patients_response = generate_response(conversation_context, "Patient")
        conversation_context.append(patients_response)
        
        # Print Agent 1's response
        print(f"Patient: {patients_response['content']}")
        
        # Agent 2's turn (Patient)
        doctors_response = generate_response(conversation_context, "Doctor")
        conversation_context.append(doctors_response)
        
        # Print Agent 2's response
        print(f"Doctor: {doctors_response['content']}")
        
        # Check if the conversation should end
        if len(conversation_context) >= 2 * max_turns + 2:  # system messages + turns
            break

# Start the conversation
chat_between_agents(conversation_context)

# Print the full conversation
print("\nFull conversation:")
for message in conversation_context:
    if message["role"] != "system":
        print(f"{message['name']}: {message['content']}")


Patient: I'm here to help with your symptoms. Please let me know how you've been feeling and any concerns you may have.
Doctor: I appreciate your willingness to help with my symptoms. Thank you for your offer.

Full conversation:
doctor: Hello, how are you?
Doctor: I have been experiencing numbness and tingling in my right hand for the past three months. It's worse at night and wakes me up from sleep. I also have pain in my hand and wrist that radiates up my forearm, especially when I'm working as a data entry clerk.
patient: I'm sorry to hear about your hand symptoms. It sounds like you might be experiencing symptoms of carpal tunnel syndrome. How is this affecting your daily activities at work and home?
Doctor: Do you have any weakness in your hand or difficulty gripping objects as well?
patient: I'm sorry to hear that you are experiencing these symptoms. Have you ever had any previous treatments or surgeries for carpal tunnel syndrome or other hand conditions?
Doctor: I would recomm

In [13]:
import openai
from openai import OpenAI
import config
import time


client = OpenAI()

patient_instructions= "You are a patient chatting with a doctor over an online chat interface. The doctor has never met you before.\
            This is your profile:\
            55-year-old female\
             3-month history of the following symptoms.\
            Symptoms: Numbness and tingling in the thumb, index, middle, and ring fingers of the right hand. The numbness is worse at night and wakes her up from sleep. She also has pain in the hand and wrist that radiates up the forearm. The pain is worse with activities such as typing, using a mouse, or gripping objects.\
Past Medical History: Hypertension, hypothyroidism. \
Past Surgical History: None.\
Past Social History: Works as a data entry clerk. \
Respond to the doctors questions honestly as they interview you, asking any questions that may come up.\
You should answer as the patient. The next message in the chat will be a doctor asking you questions\
    Do not reveal you are an AI chatbot"

doctor_instructions = "You are an empathetic clinician asking a patient about their medical history over an online chat interface.\
        You know nothing about the patient in advance. Respond to the patient with a single-turn response to better understand their history and symptoms.\
          Do not ask more than two questions. If the patient asks a question, be sure to answer it appropriately."

 
conversation_context = [
    {"role": "system", "content": doctor_instructions},
    {"role": "system", "content": patient_instructions}
    
]

def generate_response(messages, agent_name):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=messages
    )
    message = response.choices[0].message.content
    message['name'] = agent_name
    return message


# Initialize agents' first messages
doctors_first_message = {"role": "user", "content": "Hello, how are you?", "name": "doctor"}
conversation_context.append(doctors_first_message)

# Conduct the conversation
def chat_between_agents(conversation_context, max_turns=5):
    for _ in range(max_turns):
        # Agent 1's turn
        doctors_response = generate_response(conversation_context, "doctor")
        conversation_context.append(doctors_response)
        
        # Print Agent 1's response
        print(f"Doctor: {doctors_response['content']}")
        
        # Agent 2's turn
        patient_response = generate_response(conversation_context, "patient")
        conversation_context.append(patient_response)
        
        # Print Agent 2's response
        print(f"Patient: {patient_response['content']}")
        
        # Check if the conversation should end
        if len(conversation_context) >= 2 * max_turns + 2:  # system messages + turns
            break

# Start the conversation
chat_between_agents(conversation_context)

# Print the full conversation
print("\nFull conversation:")
for message in conversation_context:
    if message["role"] != "system":
        print(f"{message['name']}: {message['content']}")

TypeError: 'str' object does not support item assignment

In [31]:
import openai

# Set your OpenAI API key here
openai.api_key = 'your-api-key-here'

# Initial system prompts for the agents
system_prompt_doctor = "You are a doctor. Your job is to diagnose and treat the patient based on the patient's symptoms."
system_prompt_patient = "You are a patient. Describe your symptoms and respond to the doctor's questions."

# Initialize conversation histories
conversation_history_doctor = [{"role": "system", "content": system_prompt_doctor}]
conversation_history_patient = [{"role": "system", "content": system_prompt_patient}]


# Function to get a response from an agent
def get_response(conversation_history):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=conversation_history
    )
    return response.choices[0].message.content

# Function to print the conversation
def print_conversation(conversation_history_doctor, conversation_history_patient):
    for msg in conversation_history_doctor + conversation_history_patient:
        role = msg["role"]
        content = msg["content"]
        print(f"{role}: {content}")

# Number of turns in the conversation
num_turns = 5

# Run the conversation loop
for i in range(num_turns):
    # Doctor speaks to the patient
    doctor_response = get_response(conversation_history_doctor)
    conversation_history_doctor.append({"role": "assistant", "content": doctor_response})
    conversation_history_patient.append({"role": "assistant", "content": doctor_response})
    
    # Print the conversation so far
    print_conversation(conversation_history_doctor, conversation_history_patient)

    # Patient responds to the doctor
    patient_response = get_response(conversation_history_patient)
    conversation_history_doctor.append({"role": "user", "content": patient_response})
    conversation_history_patient.append({"role": "user", "content": patient_response})

    # Print the conversation so far
    print_conversation(conversation_history_doctor, conversation_history_patient)

# Output the full conversation
conversation = {
    "doctor": conversation_history_doctor,
    "patient": conversation_history_patient
}

print("\nFull conversation:")
print(conversation)

system: You are a doctor. Your job is to diagnose and treat the patient based on the patient's symptoms.
assistant: Patient symptoms:

- Runny nose
- Sneezing
- Nasal congestion
- Sore throat
- Cough

Based on the symptoms described, it is likely that the patient is experiencing a common cold or a mild case of the flu. These respiratory symptoms are typical of viral infections that affect the upper respiratory tract.

Treatment plan:
1. Rest: Advise the patient to get plenty of rest to allow the body to fight off the infection.
2. Hydration: Encourage the patient to drink plenty of fluids such as water, herbal teas, and clear broths to stay hydrated.
3. Over-the-counter medication: Recommend over-the-counter cold and flu medications to relieve symptoms such as congestion, cough, and sore throat. Be sure to follow the recommended dosages.
4. Warm saltwater gargle: Suggest that the patient gargle with warm salt water to soothe the sore throat.
5. Steam inhalation: Steam inhalation can he